# 不均衡データ問題

母集団数に対して非常に少数の分類が含まれているとき、それを識別できない問題。

In [ ]:
# 必要なライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

# データの生成 (3クラスの不均衡データ)
features, labels = make_blobs(n_samples=[500, 500, 3], centers=[[1, 1, 1], [0, 0, 0], [1, 1, 5]], random_state=42, n_features=3, cluster_std=0.1)

# 3Dプロットの作成
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# 各クラスのデータをプロット
ax.scatter(features[labels == 0][:, 0], features[labels == 0][:, 1], features[labels == 0][:, 2], label='Class 0', alpha=0.6)
ax.scatter(features[labels == 1][:, 0], features[labels == 1][:, 1], features[labels == 1][:, 2], label='Class 1', alpha=0.6)
ax.scatter(features[labels == 2][:, 0], features[labels == 2][:, 1], features[labels == 2][:, 2], label='Class 2', alpha=0.6)

ax.set_title('3D Imbalanced Data')
ax.legend()

plt.show()


上の図からも、class=2がclass=0やclass=1に対して数が非常に少ないことがわかる。

以下では、PCAなどの解析ツールによって、このマイナー分類がしっかり分離できるかどうかを確認する。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.decomposition import PCA
from sklearn.manifold import MDS, TSNE
from umap import UMAP

methods = ['PCA', 'MDS', 't-SNE', 'UMAP']
datasets = []

# PCA
datasets.append(PCA(n_components=2).fit_transform(features))

# MDS
datasets.append(MDS(n_components=2, random_state=42, normalized_stress='auto').fit_transform(features))

# t-SNE
datasets.append(TSNE(n_components=2, random_state=42).fit_transform(features))

# UMAP
datasets.append(UMAP(n_components=2, random_state=42, n_jobs=1).fit_transform(features))

# 可視化
fig, axs = plt.subplots(2, 2, figsize=(10, 10))  # 正方形のサブプロットを生成
axs = axs.ravel()

for i, (data, title) in enumerate(zip(datasets, methods)):
    axs[i].scatter(data[labels == 0][:, 0], data[labels == 0][:, 1], label='Class 0', alpha=0.6)
    axs[i].scatter(data[labels == 1][:, 0], data[labels == 1][:, 1], label='Class 1', alpha=0.6)
    axs[i].scatter(data[labels == 2][:, 0], data[labels == 2][:, 1], label='Class 2', alpha=0.6)
    axs[i].set_title(title)
    axs[i].legend()
    
    # スケールを揃える
    axs[i].set_aspect('equal', 'box')
    
    # 平行移動してマップを図の中央に配置
    x_center = (data[:, 0].min() + data[:, 0].max()) / 2
    y_center = (data[:, 1].min() + data[:, 1].max()) / 2
    half_range = max(data[:, 0].max() - data[:, 0].min(), data[:, 1].max() - data[:, 1].min()) / 2
    axs[i].set_xlim(x_center - 1.5*half_range, x_center + 1.5*half_range)
    axs[i].set_ylim(y_center - 1.5*half_range, y_center + 1.5*half_range)

plt.tight_layout()
plt.show()


上の結果から、PCAやMDSでは、マイナー分類(class=2)が識別できていることが分かる。
一方、t-SNEとUMAPでは、マイナー分類の識別は非常に困難であることが確認できた。

同様の問題は、多くの機械学習モデルで生じる。
（故障や異常を予測するモデルを構築するときに、正常データに比べて異常データが少ないと、この問題に直面し、適当なモデル構築が不能となる。）

## こうした「不均衡データ問題」の影響が少ない解析手法による結果

In [ ]:
import toorpia.utils as toorpia

title = 'toorPIA'
# toorPIA
data = toorpia.fit_transform(features)

# 可視化
fig, axs = plt.subplots(1, 1, figsize=(6, 5))

axs.scatter(data[labels == 0][:, 0], data[labels == 0][:, 1], label='Class 0', alpha=0.6)
axs.scatter(data[labels == 1][:, 0], data[labels == 1][:, 1], label='Class 1', alpha=0.6)
axs.scatter(data[labels == 2][:, 0], data[labels == 2][:, 1], label='Class 2', alpha=0.6)
axs.set_title(title)
axs.legend()

axs.set_aspect('equal', 'box')
    
x_center = (data[:, 0].min() + data[:, 0].max()) / 2
y_center = (data[:, 1].min() + data[:, 1].max()) / 2
half_range = max(data[:, 0].max() - data[:, 0].min(), data[:, 1].max() - data[:, 1].min()) / 2
axs.set_xlim(x_center - 1.5*half_range, x_center + 1.5*half_range)
axs.set_ylim(y_center - 1.5*half_range, y_center + 1.5*half_range)

plt.show()